#CONSTANTES

In [ ]:
import numpy as np
import numpy.random as npr
import hashlib as hl
import random

n = 256
q = (2**23)-(2**13)+1
eta = 4
k = 6
l = 5
d = 13
tau = 49
gamma1 = 2**19
gamma2 = (q-1)//32
beta = 196
omega = 55

#Fonctions aux

Basique

In [ ]:
#pour mod +-
def modular_reduction_even(r,a):
    r1 = r%a
    if(r1 <= a//2 and r1 > -a//2) :
      return r1
    if(r1>0) :
      r1 = r1 - a
    else :
      r1 = r1 + a
    return r1

def modular_reduction_odd(r,a):
    r1 = r%a
    if(r1 <= (a-1)//2 and r1 >= -(a-1)//2) :
      return r1
    if(r1>0) :
      r1 = r1 - a
    else :
      r1 = r1 + a
    return r1

def modular_reduction_plus(r,a):
  r1 = r%a
  if(r1<0) :
    r1 = r1 + a
  return r1

#calcule la norme inf d'un polynome de Rq
def NormeInfPoly(P,q=q):
  maxi = 0
  if(q%2 == 0):
    for w in P:
      maxi = max(maxi,abs(modular_reduction_even(w,q)))
  else :
    for w in P:
      maxi = max(maxi,abs(modular_reduction_odd(w,q)))
  return maxi

def NormeInfVecRq(vec, q=q):
  maxi = 0
  l = len(vec)
  for i in range(l):
    maxi = max(maxi,NormeInfPoly(vec[i],q))
  return maxi

def Power2Round(r,d=d,q=q):
  h = modular_reduction_plus(r,q)
  r0 = modular_reduction_even(r,2**d)
  s = (h-r0)//(2**d)
  return s,r0

def Decompose(r,a,q=q):
  h = modular_reduction_plus(r,q)
  if(a%2==0): r0 = modular_reduction_even(r,a)
  else : r0 = modular_reduction_odd(r,a)
  if (h-r0 == q-1) :
    r1 = 0
    r0 = r0-1
  else :
    r1 = (h-r0)//a
  return r1,r0

def HighBits(r,a,q=q):
  (r1,r0) = Decompose(r,a,q)
  return r1

def LowBits(r,a,q=q):
  (r1,r0) = Decompose(r,a,q)
  return r0

def MakeHint(z,r,a,q=q):
  r1 = HighBits(z,a,q)
  v1 = HighBits(r,a,q)
  return (r1 != v1)

def UseHint(h,r,a,q=q):
  m = (q-1)//a
  (r1,r0) = Decompose(r,a,q)
  if (h == 1 and r0 > 0) :
    return modular_reduction_plus(r1+1,m)
  if (h == 1 and r0 <= 0) :
    return modular_reduction_plus(r1-1,m)
  return r1

def BinToHex(b):
  if np.array_equal(b,np.array([0,0,0,0])):
    return '0'
  elif  np.array_equal(b,np.array([1,0,0,0])):
    return '1'
  elif  np.array_equal(b,np.array([0,1,0,0])):
    return '2'
  elif  np.array_equal(b,np.array([1,1,0,0])):
    return '3'
  elif  np.array_equal(b,np.array([0,0,1,0])):
    return '4'
  elif  np.array_equal(b,np.array([1,0,1,0])):
    return '5'
  elif  np.array_equal(b,np.array([0,1,1,0])):
    return '6'
  elif  np.array_equal(b,np.array([1,1,1,0])):
    return '7'
  elif  np.array_equal(b,np.array([0,0,0,1])):
    return '8'
  elif  np.array_equal(b,np.array([1,0,0,1])):
    return '9'
  elif  np.array_equal(b,np.array([0,1,0,1])):
    return 'a'
  elif  np.array_equal(b,np.array([1,1,0,1])):
    return 'b'
  elif  np.array_equal(b,np.array([0,0,1,1])):
    return 'c'
  elif  np.array_equal(b,np.array([1,0,1,1])):
    return 'd'
  elif  np.array_equal(b,np.array([0,1,1,1])):
    return 'e'
  elif  np.array_equal(b,np.array([1,1,1,1])):
    return 'f'

def HexToBinLilEndian(char):
  if char == '0':
    return np.array([0,0,0,0])
  elif char == '1':
    return np.array([1,0,0,0])
  elif char == '2':
    return np.array([0,1,0,0])
  elif char == '3':
    return np.array([1,1,0,0])
  elif char == '4':
    return np.array([0,0,1,0])
  elif char == '5':
    return np.array([1,0,1,0])
  elif char == '6':
    return np.array([0,1,1,0])
  elif char == '7':
    return np.array([1,1,1,0])
  elif char == '8':
    return np.array([0,0,0,1])
  elif char == '9':
    return np.array([1,0,0,1])
  elif char == 'a':
    return np.array([0,1,0,1])
  elif char == 'b':
    return np.array([1,1,0,1])
  elif char == 'c':
    return np.array([0,0,1,1])
  elif char == 'd':
    return np.array([1,0,1,1])
  elif char == 'e':
    return np.array([0,1,1,1])
  elif char == 'f':
    return np.array([1,1,1,1])

Hashing to a Ball

In [ ]:
def Packw1(w1):
  bits = ""
  nbbytes = k*128
  flat = w1.flatten()
  L = np.zeros(k*256*4, dtype=int) #6144 bits
  hexstr = ""
  for x in range(0,k*256):
    tmp = list(f'{flat[x]:0{4}b}')
    tmp2 = list(map(int, tmp))
    tmp3 = np.array(tmp2)
    tmp = BinToHex(np.flip(tmp3))
    hexstr = hexstr + tmp

  return bytes.fromhex(hexstr)

def Packt1(t1):
  bits = ""
  nbbytes = k*320
  flat = t1.flatten()
  L = np.zeros(k*256*10, dtype=int) #15360 bits
  for x in range(0,k*256*10,10):
    tmp = list(f'{flat[x//10]:0{10}b}')
    tmp2 = list(map(int, tmp))
    tmp3 = np.array(tmp2)
    L[x:x+10] = np.flip(tmp3)

  hexstr = ""
  for i in range (nbbytes*2):
    tmp = BinToHex(L[4*i:4*i+4])
    hexstr = hexstr + tmp

  return bytes.fromhex(hexstr)

def H(M1,M2):
  gfg = hl.shake_256()
  gfg.update(M1)
  gfg.update(M2)
  chat = gfg.digest(32)
  return chat

def H2(M1,M2):
  gfg = hl.shake_256()
  gfg.update(M1)
  gfg.update(M2)
  chat = gfg.digest(64)
  return chat

def SampleInBall(seed, tau=tau):
  c = np.zeros(256, dtype=int)
  gfg = hl.shake_256()
  gfg.update(seed)
  chat = gfg.digest(tau+8)
  x = np.frombuffer(chat, dtype=np.uint8)
  intrej = x[8:]
  si = np.unpackbits(x[:8]).reshape(64)[:tau]
  for i in range (256-tau,256,1):
    j = intrej[i-256+tau]
    while j > i:
      jbyte = (int(j)).to_bytes(2, byteorder='big')
      gfg.update(jbyte)
      hah = gfg.digest(1)
      j = np.frombuffer(hah, dtype=np.uint8)[0]
    c[i] = c[j]
    c[j] = (-1)**si[i-256+tau]
  return c



#GENERATION

Génération de la matrice A

---



In [ ]:
def GenererMatRqn(k1, k2, q, n):
  return npr.randint(0, q, (k1, k2, n))

def genAEasy(k=k,l=l,q=q,n=n):
  return GenererMatRqn(k,l,q,n)

def BitsToInt(bits):
  out = 0
  for bit in bits:
      out = (out << 1) | bit
  return out

def MapSeedPolyRq(seed,i,j,q=q,n=n):
  P = np.zeros(n, dtype=int)
  gfg = hl.shake_128()
  gfg.update(seed)
  gfg.update((int(256*i+j)).to_bytes(2, byteorder='little'))
  stream = gfg.hexdigest(3*n)
  for k in range (n):
    bits_Pk = np.zeros(24, dtype=int)
    substream = stream[6*k:6*k+6]
    for h in range (6) :
      bits_Pk[4*h:4*h+4] = HexToBinLilEndian(substream[h])
    bits_Pk[23] = 0
    P[k] = BitsToInt(np.flip(bits_Pk))
    while(P[k] > q - 1):
      gfg.update(int(P[k]+k).to_bytes(3, byteorder='little'))
      out = gfg.hexdigest(3)
      for h in range (6) :
        bits_Pk[4*h:4*h+4] = HexToBinLilEndian(out[h])
      bits_Pk[23] = 0
      P[k] = BitsToInt(np.flip(bits_Pk))
  return P

def ExpandA(seed, q=q,n=n,k=k,l=l):
  A = np.zeros((k,l,n), dtype=int)
  for i in range (k):
    for j in range (l):
      A[i,j] = MapSeedPolyRq(seed,i,j,q,n)
  return A

seed = npr.randint(0, 2, (n))
A = ExpandA(seed)
for i in A.flatten():
  if(i>=q) :
    print("non")


Génération de clés secrètes

---



In [ ]:
#Genere un polynome de Rq
def GenererRqn(q=q, n=n):
  return npr.randint(0, q, (n))

#Genere un polynome de Sn
def GenererSn(q=q,n=n,eta=eta):
  P = GenererRqn(eta,n)
  return P

#Generer une clé secrète de taille k
def genKey(k,n=n,q=q,eta=eta):
  s = np.zeros((k,n),dtype=int)
  for i in range (k):
    s[i] = GenererSn(q,n,eta)
  return s


def ExpandS(seed) :
  S = np.zeros((l+k,n),dtype=int)
  for i in range (l+k):
    gfg = hl.shake_256()
    gfg.update(seed)
    gfg.update((int(i)).to_bytes(2, byteorder='little'))
    nbbits = n*4
    stream = gfg.hexdigest(nbbits//8) #256 hex values
    for j in range (0,n):
      bits = np.zeros(4,dtype=int)
      bits = HexToBinLilEndian(stream[j])
      S[i][j] = BitsToInt(np.flip(bits))
      while(S[i][j] > 2*eta):
        gfg.update(int(S[i][j]).to_bytes(1, byteorder='little'))
        stream2 = gfg.hexdigest(1)#2 hex values
        bits = HexToBinLilEndian(stream2[0])
        S[i][j] = BitsToInt(np.flip(bits))
  S = eta - S
  s1 = S[:l]
  s2 = S[l:]
  return s1,s2

seed = npr.randint(0, 2, (n))
s1,s2 = ExpandS(seed)
print(s1)
print(s2)

[[ 0  4  4 ... -4 -3  4]
 [-1  3 -4 ...  1  1  3]
 [-4 -4  2 ...  1 -2 -1]
 [ 0  0 -1 ... -4  0  1]
 [-4  2  3 ...  2  0 -3]]
[[ 0  4  1 ... -3  2  2]
 [-2  2 -2 ...  1  3 -1]
 [ 2 -2  0 ... -2 -2  0]
 [-2 -3  1 ... -1  3  0]
 [ 0 -3  3 ...  4 -1  1]
 [ 0 -4  2 ...  1  2  3]]


Calcule de t : produit matrice vecteur

In [ ]:
def ProdPoly(P,Q,n=n,q=q):
  pp = np.zeros_like(P)
  for i in range (P.size):
    for j in range (P.size):
      pp[(i+j)%n] += (P[i]*Q[j])%q
  return pp % q

def dotRq(r, s, q=q):
  n = r.shape[1]
  somme =  np.zeros(n)
  for i in range(r.shape[0]):
    somme = somme + ProdPoly(r[i],s[i], n, q)
  return somme % q

def prodRqMatVec(A, s, q=q):
  As = np.zeros((A.shape[0], s.shape[1]),dtype=int)
  n = A.shape[len(A.shape)-1]
  for i in range(A.shape[0]):
    As[i, :] = dotRq(A[i,:,:], s, q)
  return As

Génération pk et sk

In [ ]:
def keygen():
  seed = npr.randint(0, 2, (n))
  gfg = hl.shake_256()
  gfg.update(seed)
  ultraseed = gfg.digest(128)
  seedA = ultraseed[:32]
  seedS = ultraseed[32:96]
  K = ultraseed[96:]
  A = ExpandA(seedA,q,n,k,l)
  s1,s2 = ExpandS(seedS)
  t=(prodRqMatVec(A,s1)+s2)%q
  t0 = np.zeros_like(t,dtype=int)
  t1 = np.zeros_like(t,dtype=int)
  for i in range (k) :
    for j in range (n) :
      (t1[i,j], t0[i,j]) = Power2Round(t[i,j],d,q)
  tr = H(seedA, Packt1(t1))

  return seedA,s1,s2,t1,t0,tr,K


#SIGNATURE

Calcul produit polynome x vecteur Rq

In [ ]:
def ProdPolyVec(poly,vec):
  (l,n) = vec.shape
  prod = np.zeros((l, n),dtype=int)
  for i in range (l):
    prod[i] = ProdPoly(poly,vec[i])
  return prod


def ExpandMask(rhoprime, k):
  y = np.zeros((l,n),dtype=int)
  gfg = hl.shake_256()
  for i in range (l) :
    gfg.update(rhoprime)
    gfg.update((int(k+i)).to_bytes(2, byteorder='little'))
    nbbits = 20*n #5120 bit pour un polynome
    stream = gfg.hexdigest( nbbits//8 ) #1280 valeurs hexa
    bits = np.zeros(nbbits, dtype=int)
    for k in range (0,len(stream)):
      bits[4*k:4*k+4] = HexToBinLilEndian(stream[k])

    for j in range (n):
      y[i,j] = gamma1-BitsToInt(np.flip(bits[20*j:20*j+20]))
  return y

def randbytes(n):
    for _ in range(n):
        yield random.getrandbits(8)


te = npr.randint(0, 2, 512)
test = ExpandMask(te, 0)

print(test)

[[  91095  159854  209813 ...  418321    4565  262214]
 [ 448938 -322388 -132063 ... -256421   68160 -336974]
 [ 144943  224251  269560 ...  388272 -287258 -126633]
 [  76329  119584 -129855 ...   77063  350780   46688]
 [ 370351 -500396 -231546 ...  326457 -178274 -479808]]


Signature

In [ ]:
def Sign(seedA,K,tr,s1,s2,t0,M):
  print("___________________")
  (l,n) = s1.shape

  A = ExpandA(seedA,q,n,k,l)

  mu = H2(tr,M)
  nn = 0
  boole = 0
  h = np.zeros((k,n),dtype=int)
  z = np.zeros((l,n),dtype=int)
  rhoprime = H(K,mu)
  while(boole == 0) :
    y = ExpandMask(rhoprime, nn)
    w = prodRqMatVec(A,y)
    w1 = np.zeros_like(w,dtype=int)
    for i in range(w1.shape[0]):
      for j in range(w1.shape[1]):
        w1[i,j] = HighBits(w[i,j],2*gamma2)

    seedc = H(mu,Packw1(w1))
    c = SampleInBall(seedc)

    cs1 = ProdPolyVec(c,s1)
    z = (y + cs1)

    cs2 = ProdPolyVec(c,s2)
    wcs2 = (w-cs2)%q

    r0 = np.zeros_like(wcs2,dtype=int)
    for i in range(r0.shape[0]):
      for j in range(r0.shape[1]):
        r0[i][j] = LowBits(wcs2[i][j],2*gamma2)

    boole = 1
    if(NormeInfVecRq(z) >= gamma1-beta or NormeInfVecRq(r0) >= gamma2-beta) :
      print(str(nn//l)+" : z " + str(NormeInfVecRq(z)-gamma1+beta) +" | r0 "+ str(NormeInfVecRq(r0)-gamma2+beta))
      boole = 0
    else:
      ct0 = ProdPolyVec(c,t0)
      wcs2cto = (w-cs2+ct0)%q
      for i in range (k):
        for j in range (n):
          h[i,j] = MakeHint(wcs2[i,j],wcs2cto[i,j],2*gamma2)
      if(NormeInfVecRq(ct0) >= gamma2 or np.count_nonzero(h) > omega) :
        print(str(nn//l)+" : z " + str(NormeInfVecRq(z)-gamma1+beta) +" | r0 "+ str(NormeInfVecRq(r0)-gamma2+beta)+" | ct0 "+str(NormeInfVecRq(ct0)-gamma2)+" | h "+str(np.count_nonzero(h)-omega))
        boole = 0
    nn = nn + l
  print(w1)
  print(mu)
  print(y)
  print(z)
  print(nn//l)
  return z,seedc,h,w1


M =  bytearray(randbytes(1000000))
M2 = bytearray(randbytes(1000000))
(seedA,s1,s2,t1,t0,tr,K) = keygen()

(z,seedc,h,w1) = Sign(seedA,K,tr,s1,s2,t0,M)
(z2,seedc2,h2,w3) = Sign(seedA,K,tr,s1,s2,t0,M2)


___________________
0 : z 1 | r0 141
1 : z -64 | r0 174
2 : z -923 | r0 182
3 : z 50 | r0 42
4 : z 8 | r0 97
5 : z 139 | r0 83
6 : z 21 | r0 -340
7 : z 139 | r0 -187
[[13  9  2 ...  9  7  7]
 [10  5 15 ...  8 11  5]
 [11  6  9 ...  5 14 15]
 [ 8 12 10 ... 10  9  7]
 [11  5 12 ...  6 13 12]
 [ 2 10  4 ...  8  9 15]]
b"\xc5\x7f`\x08Gn\x1d\x87\x91\xd5ey\xff\x88\xe0\x9aC?\x88\xd4\xd6S\x9c\x0f[\xa0\xa1\t4\xfav\x15+\xa0/\xb2\xef\x83~\x07k\x8f\xb3\xe2\xe1\xcc\xadE\x02d\xb2o\x97\xe5n\xab\x1b\x8db\t'~\x02+"
[[ 365933  151300   -9978 ...  330942  437893 -120424]
 [  21937  -24980  236659 ...  138848  302344  -22202]
 [ 489423 -476386 -292471 ...  508518  105335  -49269]
 [-414404 -291145 -433211 ...  163456  131640  172439]
 [ 364173  172343  163848 ...  285247  172192  294058]]
[[ 365942 8531697   -9976 ... 8711352  437893 8259979]
 [  21954  -24976  236670 ... 8519256  302353  -22171]
 [8869835 -476376 8087944 ...  508520 8485750  -49227]
 [7965969 -291122 7947200 ... 8543870  131648 8552855]


#VERIFICATION


In [ ]:
def Verify(seedA,M,z,seedc,t1,h):
  A = ExpandA(seedA,q,n,k,l)
  mu = H2( H(seedA, Packt1(t1)) , M )
  c = SampleInBall(seedc)
  Az = prodRqMatVec(A,z)
  c2d = c*(2**d)
  ct1 = ProdPolyVec(c2d,t1)
  Azct12d = (Az-ct1)%q
  w2 = np.zeros_like(h,dtype=int)

  for i in range(w2.shape[0]):
    for j in range(w2.shape[1]):
      w2[i][j] = UseHint(h[i][j],(Azct12d)[i][j],2*gamma2)


  if (NormeInfVecRq(z) >= gamma1 - beta) :
    print("1 non")
  if (seedc != H(mu, Packw1(w2))) :
    print("2 non")
  if (np.count_nonzero(h) > omega) :
    print("3 non")
  return (NormeInfVecRq(z) < gamma1 - beta) and (seedc == H(mu, Packw1(w2))) and (np.count_nonzero(h) <= omega)



print(Verify(seedA,M,z,seedc,t1,h))
print(Verify(seedA,M2,z,seedc,t1,h2))
print(Verify(seedA,M2,z2,seedc2,t1,h2))


True
2 non
False
True
